In [ ]:
%run shared.ipynb

In [ ]:
cell_log = pm4py.sample_cases(log, num_cases=1000)

print(len(cell_log))

pm4py.view_performance_spectrum(
    pm4py.convert_to_event_log(cell_log), ["created", "not_planned"]
)

In [ ]:
num_cases = 50

# Filter out events where org:resource is "Ghost"
cell_log = attributes_filter.apply(
    pm4py.sample_cases(
        pm4py.filter_time_range(
            filter_bot,
            "2024-01-01 00:00:00",
            "2025-01-31 00:00:00",
            mode="traces_contained",
        ),
        num_cases=num_cases,
    ),
    values={"Ghost"},
    parameters={
        attributes_filter.Parameters.ATTRIBUTE_KEY: "org:resource",
        attributes_filter.Parameters.POSITIVE: False,  # Exclude events with these values
    },
)

cell_log = pm4py.convert_to_dataframe(cell_log)
cell_log = cell_log[
    ~cell_log["author_association"].str.contains("bot", case=False, na=False)
]

# The higher the factor, the fewer the points
factor = num_cases * 1000
weight_threshold = 1 / factor

variant = sna_algorithm.Variants.HANDOVER_LOG
sna_values = sna_algorithm.apply(cell_log, variant=variant)
gviz_handover = sna_vis.apply(
    sna_values,
    variant=sna_vis.Variants.PYVIS,
    parameters={
        sna_vis.Variants.PYVIS.value.Parameters.WEIGHT_THRESHOLD: weight_threshold
    },
)


variant = sna_algorithm.Variants.WORKING_TOGETHER_LOG
sna_values = sna_algorithm.apply(cell_log, variant=variant)
gviz_working_together = sna_vis.apply(
    sna_values,
    variant=sna_vis.Variants.PYVIS,
    parameters={
        sna_vis.Variants.PYVIS.value.Parameters.WEIGHT_THRESHOLD: weight_threshold
    },
)

# variant = sna_algorithm.Variants.SUBCONTRACTING_LOG
# sna_values = sna_algorithm.apply(cell_log, variant=variant)
# gviz_subcontracting = sna_vis.apply(
#     sna_values,
#     variant=sna_vis.Variants.PYVIS,
#     parameters={
#         sna_vis.Variants.PYVIS.value.Parameters.WEIGHT_THRESHOLD: weight_threshold // 100
#     },
# )

# weight_threshold = 1 - 1 / (factor * 100000)
# variant = sna_algorithm.Variants.JOINTACTIVITIES_LOG
# sna_values = sna_algorithm.apply(cell_log, variant=variant)
# gviz_joint_activities = sna_vis.apply(
#     sna_values,
#     variant=sna_vis.Variants.PYVIS,
#     parameters={
#         sna_vis.Variants.PYVIS.value.Parameters.WEIGHT_THRESHOLD: weight_threshold
#     },
# )


# sna_vis.view(gviz_handover, variant=sna_vis.Variants.PYVIS)
# sna_vis.view(gviz_working_together, variant=sna_vis.Variants.PYVIS)
# sna_vis.view(gviz_subcontracting, variant=sna_vis.Variants.PYVIS)
# sna_vis.view(gviz_working_together, variant=sna_vis.Variants.PYVIS)

if SAVE_VIS:
    sna_vis.save(
        gviz_handover, variant=sna_vis.Variants.PYVIS, dest_file="handover.html"
    )
    sna_vis.save(
        gviz_working_together,
        variant=sna_vis.Variants.PYVIS,
        dest_file="working_together.html",
    )
    # sna_vis.save(
    #     gviz_subcontracting,
    #     variant=sna_vis.Variants.PYVIS,
    #     dest_file="subcontracting.html",
    # )
    # sna_vis.save(
    #     gviz_joint_activities,
    #     variant=sna_vis.Variants.PYVIS,
    #     dest_file="joint_activities.html",
    # )

In [ ]:
cell_log = filter_top_variants_log

petri_net, initial_marking, final_marking = pm4py.discover_petri_net_inductive(
    cell_log, noise_threshold=noise_threshold
)
pm4py.view_petri_net(petri_net, initial_marking, final_marking)

parameters_tbr = {
    token_based_replay_algorithm.Variants.TOKEN_REPLAY.value.Parameters.DISABLE_VARIANTS: True,
    token_based_replay_algorithm.Variants.TOKEN_REPLAY.value.Parameters.ENABLE_PLTR_FITNESS: True,
}
replayed_traces, place_fitness, trans_fitness, unwanted_activities_from_replay = (
    token_based_replay_algorithm.apply(
        log, petri_net, initial_marking, final_marking, parameters=parameters_tbr
    )
)

formatted_unwanted_activities = {}
for trace in pm4py.convert.convert_to_event_log(log):
    for event in trace:
        activity_name = event["concept:name"]

        if activity_name in unwanted_activities_from_replay:
            if activity_name not in formatted_unwanted_activities:
                formatted_unwanted_activities[activity_name] = []
            formatted_unwanted_activities[activity_name].append(trace)


act_diagnostics = diagnose_from_notexisting_activities(
    log, formatted_unwanted_activities
)

sorted_items = sorted(
    act_diagnostics.items(), key=lambda x: x[1]["relative_throughput"], reverse=True
)

# Print the sorted elements
for key, value in sorted_items:
    print(f"{key}: {value}")